In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np


In [183]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Correlation Matrix

In [184]:
pass_id = test["PassengerId"]
data.drop(columns="PassengerId")

#get correlations of each features in dataset
corrmat = data.corr()
top_corr_features = corrmat.index

#print(corrmat)
#plt.figure(figsize=(20,20))

#plot heat map
#g=sns.heatmap(data[top_corr_features].corr(),annot=True,cmap="RdYlGn")

# Clean data

In [192]:
avg = np.mean(data["Age"])
data["Age"] = data["Age"].fillna(value = avg)

avg = np.mean(test["Age"])
test["Age"] = test["Age"].fillna(value = avg)

print(avg)
avg = np.mean(test["Pclass"])
test["Pclass"] = test["Pclass"].fillna(value = avg)



30.272590361445815


In [186]:
Y= np.array(data["Survived"])
X = pd.DataFrame([data["Sex"], data["Age"], data["Pclass"]]).T

Ytest= np.array(data["Survived"])
Xtest = pd.DataFrame([test["Sex"], test["Age"], test["Pclass"]]).T

#label encoding
X['Sex']= X['Sex'].astype('category')
X['Sex'] = X['Sex'].cat.codes

#label encoding
Xtest['Sex']= Xtest['Sex'].astype('category')
Xtest['Sex'] = Xtest['Sex'].cat.codes

#X= pd.get_dummies(pd.DataFrame([data["Sex"], data["Age"], data["Fare"]]).T)
#X = pd.get_dummies(data, columns=['Sex'], prefix = ['Sex'])

# Linear Discriminant Analysis (LDA)

In [187]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings
warnings.filterwarnings("error")

In [197]:
errs=[]
nsplits=100
skipped=0

for j in range(nsplits):
  try:
    XTRAIN, XTEST, YTRAIN, YTEST=train_test_split(X,Y)
    LDA = LinearDiscriminantAnalysis(solver = "lsqr", shrinkage = 'auto')
    LDA.fit(XTRAIN,YTRAIN)
    YP=LDA.predict(XTEST)
    errs.append(1-accuracy_score(YTEST,YP))
  except UserWarning:
    skipped+=1
    continue
print("LDA mean error=%7.5f std=%7.5f" %(np.mean(errs),np.std(errs)))
print(skipped,"train/test splits had to be skipped because of Normalization Errors")

LDA mean error=0.20933 std=0.02376
0 train/test splits had to be skipped because of Normalization Errors


# Prediction with test.csv

In [177]:
YP = LDA.predict(Xtest)

#print(np.array(pass_id),YP);

# Save to result.csv

In [179]:
df = pd.DataFrame({"PassengerId":np.array(pass_id),
                  "Survived": YP})
df.to_csv("result.csv", sep =",",index = False)